In [ ]:
import json
import os
import sys
import traceback

import folium
import pandas as pd
from logic.src.pipeline.simulations.loader import load_simulator_data

from .notebook_setup import setup_home_directory

NOTEBOOK_NAME = "generate_maps"
home_dir = setup_home_directory(NOTEBOOK_NAME)

data_dir = os.path.join(home_dir, "data", "wsr_simulator")
graph_dir = os.path.join(data_dir, "bins_selection")
coords_dir = os.path.join(data_dir, "coordinates")
output_dir = os.path.join(data_dir, "bins_selection", "maps")
try:
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(graph_dir, exist_ok=True)
    os.makedirs(coords_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
except Exception:
    traceback.print_exc(file=sys.stdout)

In [7]:
def plot_each_route_separately(
    json_path,
    coords_xlsx_path,
    output_dir,
    marker_color="gray",
    deposito_lat=None,
    deposito_lon=None,
    name="rota",
):
    def _get_coords_from_index(idx):
        row = df_coords.iloc[idx]
        lat = float(str(row["LAT"]).replace(",", "."))
        lon = float(str(row["LNG"]).replace(",", "."))
        return lat, lon

    with open(json_path, "r") as f:
        routes = json.load(f)

    df_coords = pd.read_excel(coords_xlsx_path)
    df_coords.columns = [col.strip().upper() for col in df_coords.columns]
    for idx, route in enumerate(routes):
        # Define the center of the map
        if deposito_lat and deposito_lon:
            map_center = [deposito_lat, deposito_lon]
        elif route:
            lat, lon = _get_coords_from_index(route[0])
            map_center = [lat, lon]
        else:
            map_center = [0, 0]

        m = folium.Map(location=map_center, zoom_start=13)

        # Coordinates of the depot
        if deposito_lat and deposito_lon:
            folium.Marker(
                location=[deposito_lat, deposito_lon],
                popup="Depósito",
                icon=folium.Icon(color="green", icon="home"),
            ).add_to(m)

        points = []
        for excel_index in route:
            try:
                lat, lon = _get_coords_from_index(excel_index)
                points.append((lat, lon))
                folium.CircleMarker(
                    location=(lat, lon),
                    radius=6,
                    color=marker_color,
                    fill=True,
                    fill_opacity=0.7,
                    popup=f"Array {excel_index} (ID_coordinate: {df_coords.iloc[excel_index]['ID']})",
                ).add_to(m)
            except Exception:
                print(
                    f"Índice {excel_index} fora do range do arquivo ({len(df_coords)} linhas). Ignorado."
                )
        # if points:
        #     folium.PolyLine(points, color='blue', weight=2, opacity=0.8).add_to(m)
        #     # Linha do depósito para o primeiro ponto
        #     if deposito_lat and deposito_lon:
        #         folium.PolyLine([[deposito_lat, deposito_lon], points[0]], color='blue', weight=3, opacity=0.5).add_to(m)
        #         # Linha do último ponto de volta ao depósito
        #         folium.PolyLine([points[-1], [deposito_lat, deposito_lon]], color='blue', weight=1, opacity=0.5).add_to(m)

        outpath = os.path.join(output_dir, f"{name}{idx + 1}.html")
        m.save(outpath)
        print(f"Mapa salvo em: {outpath}")
    return


def plot_route_with_bins(
    bins,
    coords_xlsx_path,
    output_dir,
    marker_color="gray",
    deposito_lat=None,
    deposito_lon=None,
    name="rota",
):
    df_coords = pd.read_excel(coords_xlsx_path)
    df_coords.columns = [col.strip().upper() for col in df_coords.columns]

    # Função para pegar as coordenadas com base no valor do ID (bin)
    def get_coords_from_bin_id(bin_id):
        row = df_coords[df_coords["ID"] == bin_id]
        if not row.empty:
            lat = float(str(row["LAT"].values[0]).replace(",", "."))
            lon = float(str(row["LNG"].values[0]).replace(",", "."))
            return lat, lon
        else:
            return None, None  # Retorna None se não encontrar o ID

    # Definindo o centro do mapa
    if deposito_lat and deposito_lon:
        map_center = [deposito_lat, deposito_lon]
    else:
        map_center = [0, 0]  # Padrão caso não forneça as coordenadas do depósito

    m = folium.Map(location=map_center, zoom_start=13)

    # Adicionando o depósito no mapa
    if deposito_lat and deposito_lon:
        folium.Marker(
            location=[deposito_lat, deposito_lon],
            popup="Depósito",
            icon=folium.Icon(color="green", icon="home"),
        ).add_to(m)

    # Adicionando os bins no mapa com o número do contentor como popup
    for bin_id in bins:
        lat, lon = get_coords_from_bin_id(bin_id)
        if lat is not None and lon is not None:
            folium.Marker(
                location=(lat, lon),
                popup=f"Contentor {bin_id}",  # Aqui é o número do contentor
                icon=folium.Icon(color=marker_color),
            ).add_to(m)
        else:
            print(f"ID do bin {bin_id} não encontrado nas coordenadas. Ignorado.")

    # Salvando o mapa
    outpath = os.path.join(output_dir, f"{name}.html")
    m.save(outpath)
    print(f"Mapa salvo em: {outpath}")
    return

In [8]:
n_bins_ls = [20, 50, 100, 170]
for n_bins in n_bins_ls:
    plot_each_route_separately(
        json_path=os.path.join(graph_dir, f"graphs_{n_bins}V_1N_plastic.json"),
        coords_xlsx_path=os.path.join(coords_dir, "Coordinates.xlsx"),
        output_dir=output_dir,
        marker_color="orange",
        deposito_lat=39.1838505324,
        deposito_lon=-9.14806472054,
        name=f"graphs_{n_bins}V_1N",
    )

Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_20V_1N1.html
Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_50V_1N1.html
Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_100V_1N1.html
Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_170V_1N1.html


In [ ]:
area = "Rio Maior"
waste_type = "plastic"
data, coords = load_simulator_data(
    data_dir, number_of_bins=317, area=area, waste_type=waste_type
)
for n_bins in n_bins_ls:
    with open(
        os.path.join(data_dir, "bins_selection", f"graphs_{n_bins}V_1N_plastic.json")
    ) as json_file:
        bin_sel = json.load(json_file)

    df = data.iloc[bin_sel[0]].reset_index()
    bins = df["ID"]
    plot_route_with_bins(
        bins=bins,
        coords_xlsx_path=os.path.join(coords_dir, "Coordinates_unique.xlsx"),
        output_dir=output_dir,
        marker_color="orange",
        deposito_lat=39.1838505324,
        deposito_lon=-9.14806472054,
        name=f"graphs_{n_bins}V_1N",
    )

Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_20V_1N.html
Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_50V_1N.html
Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_100V_1N.html
Mapa salvo em: /home/pkhunter/Repositories/wsmart_route/data/wsr_simulator/bins_selection/maps/graphs_170V_1N.html
